In [19]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests

load_dotenv()
av_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

#### Function to structure the data

In [20]:
def structure_data(data):
    """
    Converts raw Alpha Vantage crypto daily data into a sorted closing price series.
    """
    df = pd.DataFrame.from_dict(data["Time Series (Digital Currency Daily)"], orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.astype(float).sort_index()
    return df["4. close"]

#### Function to fetch multiple tickers data


In [21]:
tickers = ["ETH", "BTC","USDT","XRP"]

In [22]:
# def fetch_data(tickers):
#     """
#     Fetch closing prices for multiple tickers and combine them into one DataFrame.
#     Each column represents one ticker.
#     """
#     market = "USD"
#     combined_df = pd.DataFrame()

#     for ticker in tickers:
#         url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market={market}&apikey={av_api_key}"
#         r = requests.get(url)
#         data = r.json()

#         # Extract and structure the series
#         closing_price_series = structure_data(data)
#         closing_price_series.name = ticker  # Set the column name

#         # Combine using outer join to align by date
#         combined_df = pd.concat([combined_df, closing_price_series], axis=1)

#     return combined_df

# # Example usage
# df = fetch_data(tickers)
# df.head()

In [23]:
# df.to_csv("Major_crypto_closing_price.csv")

In [24]:
def fetch_single_ticker(ticker):
    """
    Fetch daily crypto data, keep only open, high, low, close, volume.
    Automatically convert '1. open' → 'open', etc.
    """
    market = "USD"
    url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market={market}&apikey={av_api_key}"
    
    r = requests.get(url)
    data = r.json()

    ts = data.get("Time Series (Digital Currency Daily)", {})
    df = pd.DataFrame(ts).T

    # Convert index to datetime
    df.index = pd.to_datetime(df.index)

    # Select exact fields
    df = df[["1. open", "2. high", "3. low", "4. close", "5. volume"]]

    # Rename to standard names (remove numbers + dot + space)
    df.columns = df.columns.str.replace(r"^\d+\.\s*", "", regex=True)

    # Convert all to float
    df = df.astype(float)

    return df.sort_index()


In [25]:
ticker_name = "BTC"

In [26]:
df = fetch_single_ticker(ticker_name)

In [27]:
df.head()

,open,high,low,close,volume
2024-12-07,99885.37,100583.27,99012.73,99929.32,4774.350878
2024-12-08,99919.20,101435.05,98729.66,101174.99,4230.352252
2024-12-09,101175.00,101290.73,94220.80,97324.81,20103.050735
2024-12-10,97330.66,98338.17,94273.38,96660.76,20620.538775
2024-12-11,96660.75,101984.98,95722.79,101202.11,21190.237533


In [28]:
df.to_csv(f"{ticker_name}.csv")